In [60]:
import pandas as pd
import numpy as np
from dash import dash, html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import altair as alt

df = pd.read_csv("HIV.csv", low_memory=False)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40540 entries, 0 to 40539
Data columns (total 22 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        40540 non-null  object 
 1   REF_AREA:Geographic area                                                        40540 non-null  object 
 2   INDICATOR:Indicator                                                             40540 non-null  object 
 3   SEX:Sex                                                                         40540 non-null  object 
 4   TIME_PERIOD:Time period                                                         40540 non-null  object 
 5   OBS_VALUE:Observation Value                                                     40540 non-null  object 
 6   UNIT_MULTIPLIE

Pivot the dataframe to have each indicator on seperate column and remove the string before the column symbol on each column name \
Since we are only going to look at the full picture of each country in terms of gender, we will only keep the rows where the Sex column equals to Total.

In [62]:
df_new = df.pivot_table(values='OBS_VALUE:Observation Value', index=['REF_AREA:Geographic area', 'SEX:Sex', 'TIME_PERIOD:Time period', 'UNIT_MEASURE:Unit of measure'], columns='INDICATOR:Indicator', dropna=True, aggfunc='first')
df_new.reset_index(inplace=True)
df_new.columns = df_new.columns.str.split(':').str[-1].str.strip()
df_new['Geographic area'] = df_new['Geographic area'].str.split(':').str[-1].str.strip()
df_new['Sex'] = df_new['Sex'].str.split(':').str[-1].str.strip()
df_new['Unit of measure'] = df_new['Unit of measure'].str.split(':').str[-1].str.strip()
df_new = df_new[df_new['Sex'] == 'Total']

Check if there are any empty columns 

In [66]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13892 entries, 94 to 27616
Data columns (total 21 columns):
 #   Column                                                                                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                                                                                     --------------  ----- 
 0   Geographic area                                                                                                                                                            13892 non-null  object
 1   Sex                                                                                                                                                                        13892 non-null  object
 2   Time period                                                                              

Since column 16-20 are almost fully empty, we will remove them from the dataframe 

In [42]:
df_new = df_new.iloc[:, 0:16]

Update the data types for variables

In [59]:
## Convert time period from string to datetime
df_new['Time period'] = pd.to_datetime(df_new['Time period'], format='%Y-%m-%d')

## Convert fully numeric indicators into numeric type 
def is_numeric(column):
    try:
        pd.to_numeric(column)
        return True
    except ValueError:
        return False
numeric_columns = df_new.iloc[:, 4:16].apply(is_numeric)
numeric_column_names = numeric_columns[numeric_columns].index
df_new[numeric_column_names] = df_new[numeric_column_names].apply(pd.to_numeric)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13892 entries, 94 to 27616
Data columns (total 16 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype         
---  ------                                                                                                                                                           --------------  -----         
 0   Geographic area                                                                                                                                                  13892 non-null  object        
 1   Sex                                                                                                                                                              13892 non-null  object        
 2   Time period                                                                                      

In [44]:
df_new

INDICATOR:Indicator,Geographic area,Sex,Time period,Unit of measure,"Estimated rate of annual AIDS-related deaths (per 100,000 population)","Estimated incidence rate (new HIV infection per 1,000 uninfected population)",Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART),Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes,Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS,Per cent of pregnant women living with HIV receiving lifelong ART,Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART),Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine),Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT),Mother-to-child HIV transmission rate
94,Afghanistan,Total,2000-01-01,Number,NaN,NaN,0,NaN,NaN,1100000,2000,NaN,NaN,NaN,NaN,NaN
95,Afghanistan,Total,2000-01-01,"Rate per 100,000 of Population",0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Afghanistan,Total,2000-01-01,"Rate per 1,000 of Population",NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Afghanistan,Total,2000-01-01,Rate per 100 of Population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.70
98,Afghanistan,Total,2001-01-01,Number,NaN,NaN,0,NaN,NaN,1120000,2300,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27612,Zimbabwe,Total,2022-01-01,Number,NaN,NaN,51500,NaN,47900,830000,490000,NaN,43900,NaN,43900,NaN
27613,Zimbabwe,Total,2022-01-01,%,NaN,NaN,NaN,92.5,NaN,NaN,NaN,84.6,NaN,84.6,NaN,NaN
27614,Zimbabwe,Total,2022-01-01,"Rate per 100,000 of Population",66.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27615,Zimbabwe,Total,2022-01-01,"Rate per 1,000 of Population",NaN,1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Tabs([
        ### First tab
        dbc.Tab([ 
            html.H1('Indicator trend'),
            html.P('With a help paragraph'),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='y-col widget',
                        value='HVA_EPI_DTH_RT: Estimated rate of annual AIDS-related deaths (per 100,000 population)',
                        options=[{'label': col, 'value': col} for col in df_new.columns[5:20]], 
                        placeholder='Choose 1 indicator...'),
                    dcc.Dropdown(
                        id='country',
                        value='Canada',
                        options=[{'label': country, 'value': country} for country in df_new['Geographic area'].unique()],
                        placeholder='Choose up to 4 countries...')]),
            ]),            
            dbc.Row([
                html.Iframe(
                        id='trend',
                        style={'border-width': '0', 'width': '100%', 'height': '400px'}),
                dcc.Slider(
                        id='year slider',
                        min=2000,
                        max=2022,
                        marks={str(year): str(year) for year in range(2000, 2023)},
                        step=1,
                        value=2022)
            ])],
            label='Indicator trend'
        ),
        ### Second tab
        dbc.Tab([html.H1('Indicator map'), 
                 html.P(" ")], 
                 label='Indicator map'),
        ### Third tab
        dbc.Tab([html.H1('Indicator summary statistics'),
                 html.P(" ")], 
                 label='Indicator summary statistics')
        ])
])

if __name__ == '__main__':
    app.run_server(debug=True)